# Introduction

## Problem statement and justification for the proposed approach.

# Libraries

#### Add requirments.txt later

# EDA

# Data preparation & Feature engineering – data pre-processing – missing values, outliers, etc.
# Feature Selection – how were the features selected based on the data analysis?

# Model Training
## selection, comparison, tuning, and analysis – consider ensembles.

## I. Logistic Regression

## II. LSTM

# Model Evaluation
##  performance measures, results, and conclusions.

## I. Logistic Regression

## II. LSTM

# Model Selection

## I. Logistic Regression

## II. LSTM

# Deployment - A discussion of either the hypothetical deployment of the model or the actual deployment of the model if it has been deployed.

# Discussion and conclusions – address the problem statement and recommendation.